In [149]:
import numpy as np
from Crypto.Util.number import getPrime, inverse
from hashlib import sha256
from random import randint
from Crypto.PublicKey import RSA
import math as m

# Generate a keypair for both the sender and receiver

In [150]:
# sender_keypair = RSA.generate(2048) #recommended 2048 key length
# receiver_keypair = RSA.generate(2048)

# pwd = b'secure_password'
# with open("sender_privkey.pem", "wb") as f:
#     data = sender_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [151]:
# pwd = b'secure_password_receiver'
# with open("receiver_privkey.pem", "wb") as f:
#     data = receiver_keypair.export_key(passphrase=pwd,
#                                 pkcs=8,
#                                 protection='PBKDF2WithHMAC-SHA512AndAES256-CBC',
#                                 prot_params={'iteration_count':131072})
#     f.write(data)

In [152]:
# with open("receiver_pubkey.pem", "wb") as f:
#     data = receiver_keypair.public_key().export_key()
#     f.write(data)

In [153]:
# with open("sender_pubkey.pem", "wb") as f:
#     data = sender_keypair.public_key().export_key()
#     f.write(data)

# Message (m) and Encryption -> M

In [154]:
msg = 'secret message'.encode('utf-8')
msg

b'secret message'

# Key generation

In [155]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

In [156]:
sender_keypair = generate_keypair()
receiver_keypair = generate_keypair()
print(sender_keypair, '\n', receiver_keypair)

((17709617542179287809936331522368912730743885388392193842264747159181409383335529408078149766331054362258465915539627484796147300245182177637815138995870263929413965582944827833925308125744725691608188651973366695668277026848185763412121676691652914916404369443526689255344479004165306344528844769641210330713665700647591886882769268266443340197143766781078729049172426102035928803193057470928243797652819274194130080882291120232746839294752256096214025093769771623226341477054931812141778878945319638844080768719068924730849800394462516300773872446719840468548659921550337919521819493730763793517178303453397152605633, 4476829754281697818824670108663071655195784413504112840597503175272937335259253997044069641668440038770832821575853569820814878808368733420463008928009761482427669294681080328154859426453521097494471969989295653953972578507823594048866764379528719234678381477007029883781398307589657987095709490282004322747083604683682110241852302818735549888630032916571764160985583730089031901

x -> encoded plaintext
y -> encrypted plaintext
h -> hashed x

# Encoding Message

In [157]:
msg = 'secret message'.encode('utf-8')
x = int.from_bytes(msg, 'big')
x

2340509926146499719798124187838309

In [158]:
hacker_msg = 'not secret'.encode('utf-8')
x_hack = int.from_bytes(hacker_msg, 'big')
x_hack

521516269522829685450100

# Encryption

In [159]:
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n)
y

1482887696998856890437822102013212200089120184526933585986161388186775893451128342879033601265169164268184440279299307307519839849167180423190759633129551971287594682144030687461746561028545875859953070224437537487214110266355715597863664952866579747120640936431655026426822972163465211876350643697127194300302009238674862449141514364180987692191807915558661525966537584345025508790068453126160069720830221635696418885854289482784664178847033039679380877064778206345383771602046144860674208413829447194180045448635227544925018618727362181307012657122854380749512673602169651467066286765119774734909131841967835859859

In [160]:
y_hack = pow(x_hack, e, n)
y_hack

22320297638408400468349063868248196655851436839424095865959679374727332491901018515465117486074112043370344440223890201883458368363148630482706492004600603499214112247853576136983291978274493841218370924394758029294150415767114536877630859019727627637555230875629525696669611379193775375445293264000779029088269992195849214482552766355780854757348272874642617586809100794760003842548635969245547003195482596919117444936408571938658656236512098556663706090003565550448443605852974759426278752604651377490720836121692120379440962151238823925962504853433539831012901015072405194266716754767159894291937176453106192137273

# Hash

In [161]:
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')
h

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Signing

In [162]:
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n)
sign 

16668531137313439926312477047353754929770193976836732944417816833988465233914881256373613922413165791628959591293892997392448490121186796144255749572766530449544037830058543073936682565293361488398616202887363544241958878872309259242155181060031907732787716483922129428704053618820463806249788914968162364735668253569782838165451426667188891843207103197871258544494654401261273823251759146446659515548534419680492994114231547492375970763970461521183295739325149380419247804406636614460469512240599094808516369518386545778182997853817030462708525735049839307195284530466420456361817379694048551985512845831575692980969

# Decryption of hash (sign) with Sender pub key

In [163]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)
decrypt_to_hash

84602538464720629197225614477881030577157746024133459066183585119910221554977

# Decryption of message

In [164]:
n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)
decrypt_to_msg

2340509926146499719798124187838309

# Hash verification 

In [165]:
msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
print(final_h.digest() == hash.digest())

True


# Get plaintext

In [166]:
decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

'secret message'

In [167]:
def generate_keypair():
    p = getPrime(1024)
    q = getPrime(1024)
    n = p * q
    phi_n = (p - 1) * (q - 1)

    while True:
        e = randint(2, phi_n - 1)
        if m.gcd(e, phi_n) == 1:
            break
    
    d = inverse(e, phi_n)

    public_key = (n, e)
    private_key = (n, d)

    return public_key, private_key

# Generating keypairs
sender_keypair = generate_keypair() #generate sender keypair
receiver_keypair = generate_keypair() #generate receiver keypair
print(sender_keypair, '\n', receiver_keypair)

#Define message x
msg = 'secret message'.encode('utf-8') #secret message
x = int.from_bytes(msg, 'big') #turn bytes to integer

# Encrypt message x with receiver public key -> ciphertext y
receiver_pub, receiver_priv = receiver_keypair
n, e = receiver_pub
y = pow(x, e, n) 

# Compute hash
hash = sha256(msg)
h = int.from_bytes(hash.digest(),'big')

#Signing 
sender_pub, sender_priv = sender_keypair
n, d = sender_priv
sign = pow(h, d, n) #signing hash with sender private key

((16824228911198804917529959313726142755421720919910559681717026935267525095392078205514205068010297303794295531851551122064760818082346820613456720705686804304221311914071650955554085922576112365169649962720712832740601644083994059523291427221568470712599745306271922593896798242047779261997618962612515665941527324464175571767939999321518577427633436676212155827345023575558294885166333047311151553170406092612842363097777102500539992522878769528520016414612509952003275882377316038133290229943017813051604522955191196146311352595339528326132124087830400578532885907195051626241506320038361685476812300225673747512251, 4166934320860582160459182766372453896645313523659528186757332182745478268884213647474774689163361401316126145852479062971741431453898949631671303910626347043097800983159366770201821567082348065811065016378314242792233531131981646141714741282928682421359259065094938599428707701105954654935297708343474859959345949096823420548230408852144497747386134074612773771709933660876948468

In [168]:
n, e = sender_pub
decrypt_to_hash = pow(sign, e, n)

n, d = receiver_priv
decrypt_to_msg = pow(y, d, n)

msg_bytes = decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big')
final_h = sha256(msg_bytes)
final_h_int = int.from_bytes(final_h.digest(), 'big')  

valid_signature = decrypt_to_hash == final_h_int
print("Signature valid?", valid_signature)

decrypt_to_msg.to_bytes((decrypt_to_msg.bit_length() + 7) // 8, 'big').decode('utf-8')

Signature valid? True


'secret message'

In [169]:
from Crypto.Cipher import DES3, AES
from Crypto.Random import get_random_bytes

def generate_3des_key():
    while True:
        key = get_random_bytes(24)
        try:
            key = DES3.adjust_key_parity(key)
        except ValueError:
            continue
        
        k1, k2, k3 = key[:8], key[8:16], key[16:]
        
        if k1 != k2 and k2 != k3 and k1 != k3:
            return key, (k1, k2, k3)

key, (k1, k2, k3) = generate_3des_key()

print("3DES key (hex):", key.hex())
print("K1:", k1.hex())
print("K2:", k2.hex())
print("K3:", k3.hex())


3DES key (hex): 8c731629c2b9df266d67026845679173d50e3bbf58df51a2
K1: 8c731629c2b9df26
K2: 6d67026845679173
K3: d50e3bbf58df51a2


In [170]:
from base64 import b64encode, b64decode
from Crypto.Util.Padding import pad, unpad

def encypt_3DES(plaintext, key):
    cipher = DES3.new(key, DES3.MODE_CBC)
    ct_bytes = cipher.encrypt(pad(plaintext, DES3.block_size)) 
    initial_vector = b64encode(cipher.iv).decode('utf-8') #initialization vector -> probabilistic method
    ct = b64encode(ct_bytes).decode('utf-8') #cipher text
    return initial_vector, ct

In [171]:
data = 'Lorem ipsum dolor sit amet, consectetur adipiscing elit,' \
'sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. ' \
'Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ' \
'ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate ' \
'velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, ' \
'sunt in culpa qui officia deserunt mollit anim id est laborum.'.encode('utf_8')
iv, ct = encypt_3DES(plaintext=data, key=key)

In [172]:
def decrypt_3DES(iv, ct, key):
    try:
        cipher = DES3.new(key, DES3.MODE_CBC, iv= b64decode(iv))
        #pt = unpad(cipher.decrypt(ct), DES3.block_size)
        pt = unpad(cipher.decrypt(b64decode(ct)), DES3.block_size)
        print("The message was: ", pt)
    except (ValueError, KeyError):
        print("Incorrect decryption")

In [173]:
decrypt_3DES(iv, ct, key)

The message was:  b'Lorem ipsum dolor sit amet, consectetur adipiscing elit,sed do eiusmod tempor incididunt ut labore et dolore magna aliqua. Ut enim ad minim veniam, quis nostrud exercitation ullamco laboris nisi ut aliquip ex ea commodo consequat. Duis aute irure dolor in reprehenderit in voluptate velit esse cillum dolore eu fugiat nulla pariatur. Excepteur sint occaecat cupidatat non proident, sunt in culpa qui officia deserunt mollit anim id est laborum.'
